# Homework 7 Template

Rixin Li & G . Besla



In [1]:
 # Make edits where instructed - look for "****", which indicates where you need to add code. 

In [2]:
# import necessary modules
# numpy provides powerful multi-dimensional arrays to hold and manipulate data
import numpy as np
# matplotlib provides powerful functions for plotting figures
import matplotlib.pyplot as plt
# astropy provides unit system and constants for astronomical calculations
import astropy.units as u
import astropy.constants as const
# import Latex module so we can display the results with symbols
from IPython.display import Latex

%matplotlib inline

# **** import CenterOfMass to determine the COM pos/vel of M33
import CenterOfMass2

# **** import the GalaxyMass to determine the mass of M31 for each component
import GalaxyMass

# M33AnalyticOrbit

In [3]:
class M33AnalyticOrbit:
    """ Calculate the analytical orbit of M33 around M31 """
    
    def __init__(): # **** add inputs
        """ this class creates a set of functions that will determine the acceleration M33
                    feels from M31 and integrate its current position and velocity forwards in time. """

        ### get the gravitational constant (the value is 4.498502151575286e-06)
        self.G = const.G.to(u.kpc**3/u.Msun/u.Gyr**2).value
        
        ### **** store the output file name
        self.filename = filename
        
        ### get the current pos/vel of M33 
        # **** create an instance of the  CenterOfMass class for M33 
        M33_COM = CenterOfMass("M33_000.txt", 2)
        # **** store the position VECTOR of the M33 COM (.value to get rid of units)
        M33_COM_p =M33_COM.COM_p(.1).value
        # **** store the velocity VECTOR of the M33 COM (.value to get rid of units)
        M33_COM_v = M33_COM.COM_v(M33_COM_p[0],M33_COM_p[1],M33_COM_p[2]).value
        
        ### get the current pos/vel of M31 
        # **** create an instance of the  CenterOfMass class for M31 
        M31_COM = CenterOfMass("M31_000.txt", 2)
        # **** store the position VECTOR of the M31 COM (.value to get rid of units)
        M31_COM_p =M31_COM.COM_p(.1).value
        # **** store the velocity VECTOR of the M31 COM (.value to get rid of units)
        M31_COM_v = M31_COM.COM_v(M31_COM_p[0],M31_COM_p[1],M31_COM_p[2])
        
        ### store the DIFFERENCE between the vectors posM33 - posM31
        # **** create two VECTORs self.r0 and self.v0 and have them be the
        # relative position and velocity VECTORS of M33
        self.r0 = M33_COM_p - M31_COM_p
        self.v0 = M33_COM_v - M31_COM_v
        
        ### get the mass of each component in M31 
        ### disk
        # **** self.rdisk = scale length (no units)
        self.rdisk = 5  #as directed in homeworkfile
        # **** self.Mdisk set with ComponentMass function. Remember to *1e12 to get the right units. Use the right ptype
        self.Mdisk = ComponentMass("M31_000.txt", 2) * 1e12 
        # from assignment 3 ,we get .12
        ### bulge
        # **** self.rbulge = set scale length (no units)
        self.rbulge = 1

        
        
        # **** self.Mbulge  set with ComponentMass function. Remember to *1e12 to get the right units Use the right ptype
        self.Mbulge = ComponentMass("M31_000.txt", 3) * 1e12
        # above value should be 0.019
        # Halo
        # **** self.rhalo = set scale length from HW5 (no units)
        self.rhalo = 62 #scale radius, assignment 5
        # **** self.Mhalo set with ComponentMass function. Remember to *1e12 to get the right units. Use the right ptype
        self.Mhalo = ComponentMass("M31_000.txt", 1) * 1e12
        # 1.921
    
    
    def HernquistAccel(self, M, r_a, r): # it is easiest if you take as an input the position VECTOR 
        #3 inputs: self, mass M, r_a, r.
        #returns: the acceleration vector from a Hernquist potential.
        """ finds gravitational acceleration induced by a Hernquist profile """
        
        ### **** Store the magnitude of the position vector
        rmag = r
        
        ### *** Store the Acceleration
        Hernacc =   -(self.G * M )/ (rmag * (r_a + rmag) ** 2) * r#follow the formula in the HW instructions
        # NOTE: we want an acceleration VECTOR so you need to make sure that in the Hernquist equation you 
        # use  -G*M/(rmag *(ra + rmag)**2) * r --> where the last r is a VECTOR 
        
        return Hernacc
    
    
    
    def MiyamotoNagaiAccel(self, M, r_d, r):# it is easiest if you take as an input a position VECTOR  r 
        #inputs are M, r_d and r
        #returns acceleration vector from a Miyamoto-Nagai profile
        """For showing acceleration vector from the Miyamoto-Nagai profile
        using an approximation that mimics the exponential disk
        profile at distances far from the disk"""
        R = np.sqrt(r[0]**2 + r[1]**2) #for initializing r in x and y directions
        self.zdisk=self.rdisk/5.0 #from pdf guide 
        B=self.rdisk+ (r[2]**2+self.zdisk**2)**1/2 #since z is last column out of directions we signify it as r[2]
        #on substituting second part of pdf here, and following the multiplication, we ge t
        M_N_P_A= - ((self.G * M * r) / ((R**2 + B**2) ** 1.5)) * np.array([1, 1, B/np.sqrt(r[2]**2 + self.zdisk**2)])
        
        ### Acceleration **** follow the formula in the HW instructions
        # AGAIN note that we want a VECTOR to be returned  (see Hernquist instructions)
        # this can be tricky given that the z component is different than in the x or y directions. 
        # we can deal with this by multiplying the whole thing by an extra array that accounts for the 
        # differences in the z direction:
        #  multiply the whle thing by :   np.array([1,1,ZSTUFF]) 
        # where ZSTUFF are the terms associated with the z direction
        
        
       
        return M_N_P_A
        # the np.array allows for a different value for the z component of the acceleration
     
    
    def M31Accel(r): # input should include the position vector, r
        #returns sum of halo, buldge and disk 3d vector of total acceleration
        """ sums all acceleration vectors from each galaxy component """

        ### Call the previous functions for the halo, bulge and disk
        haloacc= HernquistAccel(self.Mhalo,self.rhalo,r)
        bulgeacc= HernquistAccel(self.Mbulge,self.rbulge,r)
        diskacc= HernquistAccel(self.Mdisk,self.rdisk,r)
        totalacc=np.sum(np.array[haloacc,bulgeacc,diskacc])
        # **** these functions will take as inputs variable we defined in the initialization of the class like 
        # self.rdisk etc. 
            
            # return the SUM of the output of the acceleration functions - this will return a VECTOR 
        return totalacc
    
    
    
    def LeapFrog(dt, r,v): # take as input dt, r and v, which are VECTORS. Assume it is ONE vector at a time
        """to solve the orbit of M33 by integrating the equation of motion forwards
in time  """
        
        # predict the position at the next half timestep
        rhalf = r+v*(dt/2)
        
        # predict the final velocity at the next timestep using the acceleration field at the rhalf position 
        vnew = v +self.M31accel*dt
        
        # predict the final position using the average of the current velocity and the final velocity
        # this accounts for the fact that we don't know how the speed changes from the current timestep to the 
        # next, so we approximate it using the average expected speed over the time interval dt. 
        rnew = r+ (.5*v+.5*vnew)*dt
        
        return rnew ,vnew # **** return the new position and velcoity vectors
    
    
    
    def OrbitIntegration(self, t0, dt, tmax):
        """  function to loop over the LeapFrog integrator to solve the equations of motion and compute
        the future orbit of M33 for 10 Gyr into the future """

        # initialize the time to the input starting time
        t = t0
        
        # initialize an empty array of size :  rows int(tmax/dt)+2  , columns 7
        orbit = np.zeros(shape=(int(tmax/dt)+2,7))
        
        # initialize the first row of the orbit
        orbit[0] = t0, *tuple(self.r0), *tuple(self.v0)
        # this above is equivalent to 
        # orbit[0] = t0, self.r0[0], self.r0[1], self.r0[2], self.v0[0], self.v0[1], self.v0[2]
        
        
        # initialize a counter for the orbit.  
        i = 1 # since we already set the 0th values, we start the counter at 1
        
        # start the integration (advancing in time steps and computing LeapFrog at each step)
        while (t<tmax):  # as long as t has not exceeded the maximal time 
            
            # **** advance the time by one timestep, dt
            t+=dt
            # **** store the new time in the first column of the ith row
            orbit[:,0] = t
            
            # ***** advance the position and velocity using the LeapFrog scheme
            # remember that LeapFrog returns a position vector and a velocity vector  
            a,b,c=leapfrog(dt,r,v)
            # as an example, if a function returns three vectors you would call the function and store 
            # the variabl
            # ****  store the new position vector into the columns with indexes 1,2,3 of the ith row of orbit
            # TIP:  if you want columns 5-7 of the Nth row of an array called A, you would write : 
            # A[n, 5:8] 
            # where the syntax is row n, start at column 5 and end BEFORE column 8
            
            
            # ****  store the new position vector into the columns with indexes 1,2,3 of the ith row of orbit
            
            
            # **** update counter i , where i is keeping track of the number of rows (i.e. the number of time steps)
            i+=1
        
        
        # write the data to a file
        np.savetxt(self.filename, orbit, fmt = "%11.3f"*7, comments='#', 
                   header="{:>10s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}{:>11s}"\
                   .format('t', 'x', 'y', 'z', 'vx', 'vy', 'vz'))
        
        # there is no return function
        
        

In [4]:
#################################

# Plot the separtion of M31 and Me separtion of M33 and M31
t=np.linspace(0,10,1//2)
plt.plot(t, total)
# Set the title
plt.title("Orbits of the Local Group", fontsize=15)


plt.tight_layout()




NameError: name 'total' is not defined

In [ ]:
# the difference would probably occur because MW was not considered during this hw, 
#including its effects would probably make our answers look closer to the previous dataset more and more 
#as we get closer and closer to the moment of collision between 
#M31 and M33, which will significantly change properties such as speed and velocity for the given galaxy